In [1]:
# Random Forest Implementation

In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import metrics

# 1 -- Load, Choose Dataset

### Choose Datasets: Regular, Smoothed, Windowed

In [4]:
# Regular Data: All Frames, No Smoothing 
# p5_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master/p5_master.csv'
# p7_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master/p7_master.csv'
# p8_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master/p8_master.csv'
# p9_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master/p9_master.csv'

# Smoothed Data: All Frames, Smoothing of Open Face Features 
# p5_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master_Smooth/p5_master_smooth.csv'
# p7_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master_Smooth/p7_master_smooth.csv'
# p8_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master_Smooth/p8_master_smooth.csv'
# p9_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master_Smooth/p9_master_smooth.csv'

# Windowed Data: 1 Second Overlapping Windows, Feature Median + Variance in Window 
p5_file = 'G:/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master_Window/p5_master_window.csv'
p7_file = 'G:/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master_Window/p7_master_window.csv'
p8_file = 'G:/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master_Window/p8_master_window.csv'
p9_file = 'G:/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master_Window/p9_master_window.csv'

data5 = pd.read_csv(p5_file)
data7 = pd.read_csv(p7_file)
data8 = pd.read_csv(p8_file)
data9 = pd.read_csv(p9_file)

In [5]:
data = [data5, data7, data8, data9]
part = [5, 7, 8, 9]

# 2 -- Choose Feature Set

Feature Dictionary: https://docs.google.com/spreadsheets/d/1ewoVPHwW68Ins0AOVZf-0lsl_wW0_ZzuByuDiNJETBY/edit?usp=sharing

### Data Overview

In [6]:
# Main Columns
basic_cols = []
for i in data[0].columns:
    if 'of_' not in i and 'op_' not in i and 'p_' not in i and 'a_' not in i:
        basic_cols.append(i)
        
basic_cols = sorted(basic_cols)
for i in basic_cols:
    print(i)

engagement
engagement_change
engagement_var
participant
session_num
timestamp


In [7]:
# Open Face Columns

of_cols = []
for i in data[0].columns:
    if 'of_' in i:
        of_cols.append(i)
        
of_cols = sorted(of_cols)
for i in of_cols:
    print(i)

of_AU01_c
of_AU01_change
of_AU02_c
of_AU02_change
of_AU04_c
of_AU04_change
of_AU05_c
of_AU05_change
of_AU06_c
of_AU06_change
of_AU07_c
of_AU07_change
of_AU09_c
of_AU09_change
of_AU10_c
of_AU10_change
of_AU12_c
of_AU12_change
of_AU14_c
of_AU14_change
of_AU15_c
of_AU15_change
of_AU17_c
of_AU17_change
of_AU20_c
of_AU20_change
of_AU23_c
of_AU23_change
of_AU25_c
of_AU25_change
of_AU26_c
of_AU26_change
of_AU28_c
of_AU28_change
of_AU45_c
of_AU45_change
of_confidence
of_confidence_var
of_gaze_0_x
of_gaze_0_x_var
of_gaze_0_y
of_gaze_0_y_var
of_gaze_0_z
of_gaze_0_z_var
of_gaze_1_x
of_gaze_1_x_var
of_gaze_1_y
of_gaze_1_y_var
of_gaze_1_z
of_gaze_1_z_var
of_gaze_angle_x
of_gaze_angle_x_var
of_gaze_angle_y
of_gaze_angle_y_var
of_gaze_distance
of_gaze_distance_var
of_pose_Rx
of_pose_Rx_var
of_pose_Ry
of_pose_Ry_var
of_pose_Rz
of_pose_Rz_var
of_pose_Tx
of_pose_Tx_var
of_pose_Ty
of_pose_Ty_var
of_pose_Tz
of_pose_Tz_var
of_pose_distance
of_pose_distance_var
of_success
of_success_change
of_ts_success


In [8]:
# Audio Columns

a_cols = []
for i in data[0].columns:
    if 'a_' in i:
        a_cols.append(i)
        
a_cols = sorted(a_cols)
for i in a_cols:
    print(i)

a_harmonicity
a_harmonicity_var
a_intensity
a_intensity_var
a_mfcc_0
a_mfcc_0_var
a_mfcc_1
a_mfcc_1_var
a_pitch_frequency
a_pitch_frequency_var
a_pitch_strength
a_pitch_strength_var


In [9]:
# Performance Columns

p_cols = []
for i in data[0].columns:
    if 'p_' in i and 'op_' not in i:
        p_cols.append(i)
        
p_cols = sorted(p_cols)
for i in p_cols:
    print(i)

p_aptitude
p_aptitude_var
p_diff_1
p_diff_2
p_diff_3
p_diff_4
p_diff_5
p_games_change
p_games_session
p_games_total
p_mistakes_change
p_mistakes_game
p_mistakes_session
p_mistakes_total
p_no_game
p_playing_change
p_skill_EM
p_skill_NC
p_skill_OS
p_ts_attempt
p_ts_game_played
p_ts_game_start
p_ts_robot_talked


In [10]:
# For Window Only:
non_window_features = []
window_features = []
for i in data[0].columns:
    if i not in basic_cols:
        if 'change' in i or 'var' in i:
            window_features.append(i)
        else:
            non_window_features.append(i)
        
window_features = sorted(window_features)
for i in window_features:
    print(i)

a_harmonicity_var
a_intensity_var
a_mfcc_0_var
a_mfcc_1_var
a_pitch_frequency_var
a_pitch_strength_var
of_AU01_change
of_AU02_change
of_AU04_change
of_AU05_change
of_AU06_change
of_AU07_change
of_AU09_change
of_AU10_change
of_AU12_change
of_AU14_change
of_AU15_change
of_AU17_change
of_AU20_change
of_AU23_change
of_AU25_change
of_AU26_change
of_AU28_change
of_AU45_change
of_confidence_var
of_gaze_0_x_var
of_gaze_0_y_var
of_gaze_0_z_var
of_gaze_1_x_var
of_gaze_1_y_var
of_gaze_1_z_var
of_gaze_angle_x_var
of_gaze_angle_y_var
of_gaze_distance_var
of_pose_Rx_var
of_pose_Ry_var
of_pose_Rz_var
of_pose_Tx_var
of_pose_Ty_var
of_pose_Tz_var
of_pose_distance_var
of_success_change
op_num_people_var
p_aptitude_var
p_games_change
p_mistakes_change
p_playing_change


### Filter Feature Set (Optional)

In [11]:
# features_to_keep = data[0].columns

# always include basic_cols, add desired group of features 
# features_to_keep = basic_cols + of_cols + p_cols + a_cols

# for i,d in enumerate(data):
#     data[i] = d[features_to_keep]

# 3 -- Train-Test Split

In [12]:
# Helper Function: Split a Participant's Data Chronologically 

def part_split(split, split_size):
    split = split.sort_values(['session_num', 'timestamp'], ascending=[True, True])
    bogus = split['engagement']

    split_train, split_test, bogus1, bogus2 = train_test_split(split, bogus, test_size=split_size, shuffle=False)
    return split_train, split_test

In [13]:
# Function: Formulate Train-Test Split 

# train_part: an array of participants 
# test_part: single participant
# split_size: how much of test_part data to use for testing
def split(train_part, test_part, split_size):
    train_data = []
    for p in train_part:
        i = part.index(p)
        train_data.append(data[i])
    
    i = part.index(test_part)
    test_data = data[i]
    if split_size < 1:
        split_train, test_data = part_split(test_data, split_size)
        train_data.append(split_train)
    
    train_data = pd.concat(train_data, ignore_index=True, sort=True)
    
    return train_data, test_data

In [26]:
# Train-Test Split

# Excluding Participant 8
perc = 1
#train, test = split(train_part=[5,7], test_part=9, split_size=perc)
#train, test = split(train_part=[5,9], test_part=7, split_size=perc)
train, test = split(train_part=[7,9], test_part=5, split_size=perc)

# Individualized
# perc = 0.7
# train, test = split(train_part=[], test_part=7, split_size=perc)
# train, test = split(train_part=[], test_part=5, split_size=perc)
# train, test = split(train_part=[], test_part=9, split_size=perc)
# train, test = split(train_part=[], test_part=8, split_size=perc)

# 4 -- Scenario Based Modeling

- During Game / Outside of Game
- Open Face Success / Failure
- Robot Talking / Not Talking
- First 10 Minutes / After 10 Minutes

In [27]:
# Create Separate Models for Different Scenarios

# # Outside of Game
# train = train.loc[train['no_game']==1]
# test = test.loc[test['no_game']==1]

# # During Game
# train = train.loc[train['no_game']==0]
# test = test.loc[test['no_game']==0]

# # Open Face Success
# train = train.loc[train['of_success']==1]
# test = test.loc[test['of_success']==1]

# # Open Face Failure
# train = train.loc[train['of_success']==0]
# test = test.loc[test['of_success']==0]

# # Robot Talking
# train = train.loc[train['p_ts_robot_talked']==0]
# test = test.loc[test['p_ts_robot_talked']==0]

# # Robot Not Talking
# train = train.loc[train['p_ts_robot_talked']>0]
# test = test.loc[test['p_ts_robot_talked']>0]

# # First 10 Minutes of Session
# train = train.loc[train['timestamp']<=(10*60)]
# test = test.loc[test['timestamp']<=(10*60)]

# # After first 10 Minutes of Session
# train = train.loc[train['timestamp']>(10*60)]
# test = test.loc[test['timestamp']>(10*60)]

### Divide into X_train, y_train, X_test, y_test

In [28]:
y_train = train['engagement']
X_train = train.drop(columns=basic_cols)

y_test = test['engagement']
X_test = test.drop(columns=basic_cols)

In [29]:
# Sort Training Data Together 

X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)

# 5 -- Preprocessing
Standardization Recommended

### Min-Max Scaling
- x' = ( x - min(x) ) / ( max(x) - min(x) )
- => x' in [0, 1], fill NaN with -1

In [30]:
# for c in X_train.columns:
#     min_val = np.nanmin(X_train[c])
#     max_val = np.nanmax(X_train[c])
    
#     X_train[c] = (X_train[c]-min_val)/(max_val - min_val)
#     X_test[c] = (X_test[c]-min_val)/(max_val - min_val)
    
# X_train = X_train.fillna(-1)
# X_test = X_test.fillna(-1)

### Standardization (Recommended)
- x' = ( x - mean(x) ) / ( stdev(x) )
- => x' is z-score, NaN's filled with min_val

In [31]:
for c in X_train.columns:
    mean = np.nanmean(X_train[c])
    std = np.nanstd(X_train[c])
    
    X_train[c] = (X_train[c]-mean)/(std)
    X_test[c] = (X_test[c]-mean)/(std)
        
    if 'of_pose_distance' not in c:
        min_val = np.nanmin(X_train[c])

        X_train[c] = X_train[c].fillna(min_val)
        X_test[c] = X_test[c].fillna(min_val)
    else:
        max_val = np.nanmax(X_train[c])

        X_train[c] = X_train[c].fillna(max_val)
        X_test[c] = X_test[c].fillna(max_val)

# 6 -- Model!

In [32]:
# TODO: Should Try K-Folds Cross Validation to Get Best Hyper Parameters

In [33]:
from sklearn.ensemble import RandomForestClassifier

In [36]:
clf = RandomForestClassifier(n_estimators=10, max_depth=5, criterion='entropy', max_features='auto')
clf.fit(X_train.values, y_train.values)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [37]:
# Train Accuracy 

train_pred = clf.predict(X_train.values)
train_scores = clf.predict_proba(X_train.values)[:,1]
try:
    print("AUC:",roc_auc_score(y_train.values, train_scores)) 
except:
    print("AUC undefinied, only 1 class in test data")
    
print("Accuracy:",accuracy_score(y_train.values, train_pred))

AUC: 0.9460460086468699
Accuracy: 0.8700399974604787


In [38]:
pred = clf.predict(X_test.values)
scores = clf.predict_proba(X_test.values)[:,1]

# 7 -- Evaluation

In [39]:
# y_test: actual y values for test data
# scores: predicted probabilistic y values for test data
# pred: predicted binary y values for test data

In [285]:
try:
    print("AUC:",roc_auc_score(y_test, scores)) 
except:
    print("AUC undefinied, only 1 class in test data")

print("Accuracy:",accuracy_score(y_test, pred))
print(metrics.confusion_matrix(y_test, pred))
print(metrics.classification_report(y_test, pred))

AUC: 0.9063420729649934
Accuracy: 0.8153077521602031
[[ 9406  3386]
 [ 1124 10503]]
              precision    recall  f1-score   support

         0.0       0.89      0.74      0.81     12792
         1.0       0.76      0.90      0.82     11627

   micro avg       0.82      0.82      0.82     24419
   macro avg       0.82      0.82      0.81     24419
weighted avg       0.83      0.82      0.81     24419



[0.8153077521602031,
 0.9063420729649934,
 '0.89',
 '0.76',
 '0.74',
 '0.90',
 '0.81',
 '0.82']

# 8--Randomized Random Forest best params

In [41]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 42)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 'warn',
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [42]:
#re-parameterizing is a pain in the ass, so I made this variation of section 6 +7
from sklearn.model_selection import RandomizedSearchCV

In [44]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 110, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
n_estimators

[10, 21, 32, 43, 54, 65, 76, 87, 98, 110]

In [ ]:
rf_random.fit(X_train.values, y_train.values)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 12.0min


In [ ]:
rf_random.best_params_

In [ ]:
#evaluate best parameters
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [ ]:
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test.values, y_test)

In [ ]:
solutions= []

solutions.append(accuracy_score(y_test, pred))
solutions.append(roc_auc_score(y_test, scores))
solutions.append(metrics.classification_report(y_test,pred)[70:80].strip())
solutions.append(metrics.classification_report(y_test,pred)[128:133].strip())
solutions.append(metrics.classification_report(y_test,pred)[83:90].strip())
solutions.append(metrics.classification_report(y_test,pred)[136:142].strip())
solutions.append(metrics.classification_report(y_test,pred)[90:100].strip())
solutions.append(metrics.classification_report(y_test,pred)[145:152].strip())

solutions